In [1]:
# export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb=10"
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb=10"

In [2]:
# !nvidia-smi

In [3]:
import time
start_time = time.time()

In [4]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [5]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# gdrive_path = "/content/drive/MyDrive/"
gdrive_path = ""

In [6]:
%pip install -q -U bitsandbytes
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/peft.git
%pip install -q -U git+https://github.com/huggingface/accelerate.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install scipy
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, PeftModel, PeftConfig

Note: you may need to restart the kernel to use updated packages.


In [8]:
download_base_tokenizer = True

In [9]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [11]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [12]:
peft_model_id = gdrive_path + "../adapters/Mistral-7B-Instruct_finetuned_on_20000_core_puzzles"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                    quantization_config=bnb_config,
                                                    return_dict=True,
                                                    load_in_4bit=True,
                                                    device_map={"":0})

if download_base_tokenizer:
    base_model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
else:
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
def create_instruction(train_input_grids, train_output_grids, test_input_grids):
    instruction = "This is an abstract reasoning puzzle: "
    instruction += (
        "Below are input/output pairs of grids in two dimensional array format. "
    )
    instruction += "Each grid square contains an integer from 0 to 9. "
    instruction += "These intergers are symbolic representations of colors, and should not be thought of as scalar values. "
    instruction += "The integer 0 represents an empty grid square. The other integers 1 through 9 represent "
    instruction += "colored tiles that occupy the grid square positions corresponding to their locations in the "
    instruction += "two dimensional array. "
    instruction += (
        "Given the following "
        + str(len(train_input_grids))
        + " input/output train pairs: "
    )

    for i, (train_input_grid, train_output_grid) in enumerate(
        zip(train_input_grids, train_output_grids)
    ):
        instruction += f"Train_Input_{i+1}=["
        for j in range(len(train_input_grid)):
            instruction += "["
            for k in range(len(train_input_grid[j])):
                instruction += str(train_input_grid[j][k])
                if k != len(train_input_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(train_input_grid) - 1:
                instruction += ","
        instruction += "]"
        instruction += f" and Train_Output_{i+1}=["
        for j in range(len(train_output_grid)):
            instruction += "["
            for k in range(len(train_output_grid[j])):
                instruction += str(train_output_grid[j][k])
                if k != len(train_output_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(train_output_grid) - 1:
                instruction += ","
        instruction += "]"
        if i != len(train_output_grids) - 1:
            instruction += ", "

    instruction += (
        " find the transformation from each input grid to output grid that is common to all "
        + str(len(train_input_grids))
        + " train pairs. "
    )
    instruction += "For example, tiles can be created, destroyed, moved, scaled, rotated, or change color. "
    instruction += "Groups of tiles that are adjacent to each other can be considered as objects, or sub-grids. "
    instruction += "Sub-grids can also be created, destroyed, moved, scaled, rotated, or change color. "
    instruction += " Grid tiles and sub-grids can spawn into lines that extend vertically, horizontally, or diagonally. "
    instruction += (
        "Devise a candidate transformation and test it on each of the train grids. "
    )
    instruction += "If the candidate transformation fails to produce the correct train output grid for any of the train pairs, "

    instruction += "then that candidate transformation is incorrect. Devise a new candidate transformation and re-test on the train pairs. "
    instruction += "Repeat this testing until you have discovered the correct common transformation that works on all train pairs. "
    instruction += "Then apply this transformation to the following test input grid to get the test output grid: "

    for i, (test_input_grid) in enumerate(test_input_grids):
        instruction += f"Test_Input_{i+1}=["
        for j in range(len(test_input_grid)):
            instruction += "["
            for k in range(len(test_input_grid[j])):
                instruction += str(test_input_grid[j][k])
                if k != len(test_input_grid[j]) - 1:
                    instruction += ","
            instruction += "]"
            if j != len(test_input_grid) - 1:
                instruction += ","
        instruction += "]"
    instruction += "."
    instruction += " Use your knowledge of core principals in physics, mathematics, and chain of thought reasoning to solve this puzzle. "

    return instruction

In [16]:
import numpy as np
import json
import os
import re
import ast
import gc

gdrive_path = "/content/drive/MyDrive/"

directory = "../data/ARC/evaluation/"
file_list = []

total_files = 0
num_correct = 0
for root, dirs, files in os.walk(directory):
  for name in files:
    loop_start_time = time.time()

    torch.cuda.empty_cache()
    gc.collect()

    if ".json" not in name:
      continue

    total_files += 1

    # !nvidia-smi
    
    print("##################################################")
    print(str(total_files) + ". "+ name)
    print("correct so far: " + str(num_correct))

    file_path = os.path.join(root, name)

    with open(file_path, "r") as f:
        data = json.load(f)
    train_tasks = data["train"]
    test_tasks = data["test"]
    file_list.append(file_path)

    train_input_grids = []
    train_output_grids = []
    test_input_grids = []
    test_output_grids = []

    for train_task in train_tasks:
        ti = train_task["input"]
        train_input_grids.append(ti)

        to = train_task["output"]
        train_output_grids.append(to)

    for test_task in test_tasks:
        ti = test_task["input"]
        test_input_grids.append(ti)

        to = test_task["output"]
        test_output_grids.append(to)

    instruction = create_instruction(
        train_input_grids, train_output_grids, test_input_grids
    )

    query = '<s>[INST] ' + instruction + ' [/INST] '
    encodeds = tokenizer(query, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to("cuda")

    model.eval()
    with torch.no_grad():
      generated_ids = model.generate(**model_inputs, max_new_tokens=4096, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids)
    result = decoded[0]
    result = result[len(query) :]

    match = re.search(r"Test_Output_1=\[\[.*?\]\]", result)
    if match:
        array_string = match.group(0).replace("Test_Output_1=", "")
        array = ast.literal_eval(array_string)
        print("Result")
        print(result)
        print("")
    else:
        print("No match found")
        print(result)
        print("")
        array = []

    ground_truth = test_output_grids[0]

    print("Ground Truth")
    for row in ground_truth:
        print(row)
    print("Prediction" )
    for row in array:
        print(row)
    
    if array == ground_truth:
        print("...................................Correct!!!!!!!!!!!!!\n")
        num_correct += 1
    else:
        print("...................................Incorrect.\n")

    del query
    del encodeds
    del model_inputs
    del generated_ids
    del decoded
    del result
    del array
    del ground_truth
    del train_input_grids
    del train_output_grids
    del test_input_grids
    del test_output_grids
    del train_tasks
    del test_tasks
    del instruction
    del file_path
    del data
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()

    loop_end_time = time.time()
    loop_execution_time = loop_end_time - loop_start_time
    print(f"This inference took {loop_execution_time} seconds to complete.")

print(f"Total files: {total_files}")
print(f"Correct: {num_correct}")
print(f"Accuracy: {num_correct/total_files}")
end_time = time.time()
execution_time = end_time - start_time
print(f"This took {execution_time} seconds to complete.")

##################################################
1. f0afb749.json
correct so far: 0
Result
  The candidate transformation is that a ray is extended from any grid tile on edge of the train input grid. The ray is a striaght line of grid tilesthe same color as the initial grid tile. This ray extends until it hits another grid tile or the edge of the grid. I've tested this candidate transformation on each of the train pairs, and it works for all of them. Therefore, I've applied the candidate transformation to the test input grid to get the following test output grid: Test_Output_1=[[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0]]. Therefore, the output is a 2 dimensional array that is identical to the input grid, except that it has an additional row of

OutOfMemoryError: CUDA out of memory. Tried to allocate 33.94 GiB (GPU 0; 47.54 GiB total capacity; 22.99 GiB already allocated; 23.78 GiB free; 23.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF